In [4]:
from tensorflow import keras
import os
import numpy as np
import cv2

In [7]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(len(x_train))

# Define the path to the folder containing the image folders
base_path = "printed_digits_dataset/assets/"

# Initialize the x and y variables
x = []
y = []

# Loop through the folders labeled 1-10
for label in range(0, 10):
    folder_path = os.path.join(base_path, str(label))
    
    # Get the list of image files in the folder
    image_files = os.listdir(folder_path)
    
    # Loop through each image file in the folder
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        # Load the image using OpenCV
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Append the image to the x variable
        x.append(image)
        
        # Append the label to the y variable
        y.append(label)

# Convert the x variable to a numpy array
x_train = np.concatenate((x_train,np.array(x)),axis=0)

# Convert the y variable to a numpy array
y_train = np.concatenate((y_train,np.array(y)),axis=0)

# Step 1: Random Line Augmentation Function
def add_random_lines(img, num_lines=3, max_thickness=5, max_length=20):
    augmented = img.copy()
    h, w = img.shape[:2]
    
    for _ in range(num_lines):
        x1, y1 = np.random.randint(0, w), np.random.randint(0, h)
        x2, y2 = x1 + np.random.randint(-max_length, max_length), y1 + np.random.randint(-max_length, max_length)
        thickness = np.random.randint(1, max_thickness)
        color = np.random.randint(0, 256)
        cv2.line(augmented, (x1, y1), (x2, y2), color, thickness)
    
    return augmented

# Step 2: Generate Random Line Images for Label '0'
num_random_images = 2000  # Adjust this as required
random_line_images = [add_random_lines(np.zeros((28, 28), dtype=np.uint8)) for _ in range(num_random_images)]
random_line_labels = [0] * num_random_images

# Step 3: Integrate Augmented Data with Your Current Data
x_train = np.concatenate((x_train, random_line_images), axis=0)
y_train = np.concatenate((y_train, random_line_labels), axis=0)

print(len(x_train))

# Preprocess the data (e.g., normalize pixel values)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define the model architecture
mnist_model2 = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
mnist_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the data to match the input shape of the new model
x_train2 = np.expand_dims(x_train, axis=-1)
x_test2 = np.expand_dims(x_test, axis=-1)

from keras.preprocessing.image import ImageDataGenerator

# Augment data a little
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train2)

# Train the model
mnist_model2.fit(datagen.flow(x_train2, y_train, batch_size=32),
                epochs=10, validation_data=(x_test2, y_test))

60000
68299
Epoch 1/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.5024 - accuracy: 0.8442 - val_loss: 0.0403 - val_accuracy: 0.9877
Epoch 2/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.2244 - accuracy: 0.9304 - val_loss: 0.0297 - val_accuracy: 0.9903
Epoch 3/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.1683 - accuracy: 0.9494 - val_loss: 0.0266 - val_accuracy: 0.9916
Epoch 4/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.1404 - accuracy: 0.9571 - val_loss: 0.0202 - val_accuracy: 0.9934
Epoch 5/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.1256 - accuracy: 0.9621 - val_loss: 0.0269 - val_accuracy: 0.9913
Epoch 6/10
2135/2135 [==============================] - 18s 8ms/step - loss: 0.1113 - accuracy: 0.9663 - val_loss: 0.0215 - val_accuracy: 0.9920
Epoch 7/10
2135/2135 [==============================] - 19s 9ms/step - loss: 0.1021 - accuracy: 0.9689 - val_loss: 0.0

In [8]:
mnist_model2.save_weights('digit_recognizer_test.h5')